In [1]:
import pandas as pd       
import os 
import math 
import numpy as np
import matplotlib.pyplot as plt  
import IPython.display as ipd  # To play sound in the notebook
import librosa
import librosa.display

In [3]:
metadata=pd.read_csv('speakers_all.csv')
metadata.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,24.0,12.0,"koussi, senegal",balanta,balanta,male,788,senegal,True,NaN,NaN,NaN
1,18.0,10.0,"buea, cameroon",cameroon,cameroon,male,1953,cameroon,True,NaN,NaN,NaN
2,48.0,8.0,"hong, adamawa, nigeria",fulfulde,fulfulde,male,1037,nigeria,True,NaN,NaN,NaN
3,42.0,42.0,"port-au-prince, haiti",haitian,haitian,male,1165,haiti,True,NaN,NaN,NaN
4,40.0,35.0,"port-au-prince, haiti",haitian,haitian,male,1166,haiti,True,NaN,NaN,NaN


In [4]:
metadata = metadata[metadata['file_missing?'] != True]
metadata.shape

(2138, 12)

In [5]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [6]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = 'recordings/recordings/' + str(row["filename"]) + '.mp3'
    final_class_labels=row["sex"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

0it [00:00, ?it/s]C:\Users\KIIT\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
1it [00:01,  1.00s/it]C:\Users\KIIT\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
2138it [18:36,  1.92it/s]


In [7]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-283.2144, 102.64268, 12.902011, 43.4498, 5.5...",female
1,"[-252.67426, 99.67917, 0.25114948, 48.571484, ...",male
2,"[-298.53323, 111.03142, 3.1933663, 43.25243, 1...",male
3,"[-250.61511, 117.96154, 8.878023, 51.569702, 6...",male
4,"[-329.30942, 110.29973, 10.1238165, 16.642046,...",male


In [8]:
extracted_features_df.to_csv('MFCC_Extracted_features.csv', index=False)

In [9]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [10]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
y

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [12]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((1710, 40), (428, 40), (1710, 2), (428, 2))

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [14]:
input_shape = (40,)
num_labels = 2

model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [15]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [16]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved models/Gender_Classifier_ANN.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
54/54 [==============================] - 1s 5ms/step - loss: 16.1523 - accuracy: 0.5146 - val_loss: 1.7881 - val_accuracy: 0.5023

Epoch 00001: val_loss improved from inf to 1.78812, saving model to saved models\Gender_Classifier_ANN.hdf5
Epoch 2/100
54/54 [==============================] - 0s 2ms/step - loss: 5.7564 - accuracy: 0.4930 - val_loss: 0.8255 - val_accuracy: 0.4977

Epoch 00002: val_loss improved from 1.78812 to 0.82553, saving model to saved models\Gender_Classifier_ANN.hdf5
Epoch 3/100
54/54 [==============================] - 0s 2ms/step - loss: 2.9358 - accuracy: 0.5263 - val_loss: 0.6844 - val_accuracy: 0.5304

Epoch 00003: val_loss improved from 0.82553 to 0.68440, saving model to saved models\Gender_Classifier_ANN.hdf5
Epoch 4/100
54/54 [==============================] - 0s 2ms/step - loss: 1.8331 - accuracy: 0.5263 - val_loss: 0.6751 - val_accuracy: 0.7009

Epoch 00004: val_loss improved from 0.68440 to 0.67512, saving model to saved models\Gender_Classif

54/54 [==============================] - 0s 2ms/step - loss: 0.2593 - accuracy: 0.9117 - val_loss: 0.2212 - val_accuracy: 0.9206

Epoch 00037: val_loss did not improve from 0.21488
Epoch 38/100
54/54 [==============================] - 0s 2ms/step - loss: 0.2739 - accuracy: 0.9053 - val_loss: 0.2149 - val_accuracy: 0.9322

Epoch 00038: val_loss did not improve from 0.21488
Epoch 39/100
54/54 [==============================] - 0s 2ms/step - loss: 0.2555 - accuracy: 0.9135 - val_loss: 0.2152 - val_accuracy: 0.9346

Epoch 00039: val_loss did not improve from 0.21488
Epoch 40/100
54/54 [==============================] - 0s 2ms/step - loss: 0.2575 - accuracy: 0.9152 - val_loss: 0.2150 - val_accuracy: 0.9276

Epoch 00040: val_loss did not improve from 0.21488
Epoch 41/100
54/54 [==============================] - 0s 2ms/step - loss: 0.2418 - accuracy: 0.9152 - val_loss: 0.2118 - val_accuracy: 0.9393

Epoch 00041: val_loss improved from 0.21488 to 0.21178, saving model to saved models\Gender_Cl

54/54 [==============================] - 0s 2ms/step - loss: 0.1797 - accuracy: 0.9368 - val_loss: 0.2353 - val_accuracy: 0.9299

Epoch 00079: val_loss did not improve from 0.21178
Epoch 80/100
54/54 [==============================] - 0s 2ms/step - loss: 0.1915 - accuracy: 0.9404 - val_loss: 0.2487 - val_accuracy: 0.9182

Epoch 00080: val_loss did not improve from 0.21178
Epoch 81/100
54/54 [==============================] - 0s 2ms/step - loss: 0.1760 - accuracy: 0.9386 - val_loss: 0.2312 - val_accuracy: 0.9229

Epoch 00081: val_loss did not improve from 0.21178
Epoch 82/100
54/54 [==============================] - 0s 2ms/step - loss: 0.1765 - accuracy: 0.9398 - val_loss: 0.2382 - val_accuracy: 0.9229

Epoch 00082: val_loss did not improve from 0.21178
Epoch 83/100
54/54 [==============================] - 0s 2ms/step - loss: 0.1980 - accuracy: 0.9386 - val_loss: 0.2264 - val_accuracy: 0.9206

Epoch 00083: val_loss did not improve from 0.21178
Epoch 84/100
54/54 [=======================

In [17]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.927570104598999


In [18]:
model.predict_classes(X_test)

C:\Users\KIIT\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,

In [20]:
filename="recordings/recordings/english381.mp3"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)

predicted_label=model.predict_classes(mfccs_scaled_features)

prediction_class = labelencoder.inverse_transform(predicted_label) 

predicted_label, prediction_class

(array([1], dtype=int64), array(['male'], dtype='<U6'))